In [1]:
!pip install kaggle TfELM tensorflow==2.10

In [2]:
import os
import torch
import numpy as np
import pandas as pd
import torchvision as tv
import matplotlib.pyplot as plt
import plotly.express as px
import tensorflow as tf
from PIL import Image
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold, cross_val_score, train_test_split
from sklearn import preprocessing
from TfELM.Layers.KELMLayer import KELMLayer
from TfELM.Models.KELMModel import KELMModel
from TfELM.Resources.Kernel import Kernel, CombinedProductKernel
from google.colab import userdata
%matplotlib inline

In [3]:
!kaggle datasets download -d infokomalgebra/alzheption-extractor-dataset

!unzip "alzheption-extractor-dataset.zip"

Dataset URL: https://www.kaggle.com/datasets/infokomalgebra/alzheption-extractor-dataset
License(s): apache-2.0
100% 4.03G/4.05G [00:47<00:00, 204MB/s]
100% 4.05G/4.05G [00:47<00:00, 91.9MB/s]
Archive:  alzheption-extractor-dataset.zip
replace 1_test_dataset.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 1_test_features.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 1_test_labels.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 1_train_dataset.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 1_train_features.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 1_train_labels.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 2_test_dataset.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 2_test_features.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 2_test_labels.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 2_train_dataset.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 2_train_features.npz? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace 2_

In [4]:
os.remove('alzheption-extractor-dataset.zip')

In [5]:
# Hyperparameters:
num_neurons = 512
n_splits = 10
n_repeats = 1
max_data = 95

In [6]:
# Loading sample dataset from Data folder
X_train = np.load('1_train_features.npz')['train_features']
X_test = np.load('1_test_features.npz')['test_features']
y_train = np.load('1_train_labels.npz')['train_labels']
y_test = np.load('1_test_labels.npz')['test_labels']

X = np.concatenate([X_train, X_test])
y = np.concatenate([y_train, y_test])

# label 4: 95 data. so, all data will be crop in 95 data.
result = {}
for data, label in zip(X, y):
    if label not in result:
        result[label] = [data]
        continue

    if len(result[label]) == max_data:
        continue

    result[label].append(data)

X = np.concatenate(list(result.values()))
y = np.concatenate([[_]*len(result[label]) for _ in result])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape
# X.shape, y.shape, X.shape, y.shape

((456, 2048), (456,), (114, 2048), (114,))

In [7]:
np.unique(y_train, return_counts=True), np.unique(y_test, return_counts=True)

((array([0, 1, 2, 3, 4, 5]), array([76, 76, 76, 76, 76, 76])),
 (array([0, 1, 2, 3, 4, 5]), array([19, 19, 19, 19, 19, 19])))

In [8]:
X_train[0]

array([0.151683  , 0.242826  , 0.35081708, ..., 0.10494268, 0.        ,
       0.22506896], dtype=float32)

In [9]:
y_train[0]

4

In [10]:
# Label encoding and features normalization
label_encoder = LabelEncoder()
X_train_scaled = preprocessing.normalize(X_train)  # Normalize feature vectors
y_train_scaled = label_encoder.fit_transform(y_train)  # Encode class labels to numerical values

X_train_scaled.shape, y_train_scaled.shape

((456, 2048), (456,))

In [11]:
X_train_scaled[0]

array([0.00723931, 0.01158925, 0.01674329, ..., 0.00500855, 0.        ,
       0.01074177], dtype=float32)

In [12]:
y_train_scaled[0]

4

In [13]:
# Initialize a Kernel (it can be instanced as Kernel class and its subclasses like CombinedProductKernel)
# kernel = CombinedProductKernel([Kernel("rational_quadratic"), Kernel("exponential")])

list_kernel = ['rbf', 'exponential', 'rational_quadratic']
list_func = ['mish', 'leaky_relu', 'relu', 'swish', 'silu']

In [14]:
result = []
for name_kernel in list_kernel:
    for name_func in list_func:
        kernel = Kernel(name_kernel)

        # Initialize a Kernel Extreme Learning Machine (KELM) layer
        layer = KELMLayer(kernel, name_func)

        # Initialize a Kernel Extreme Learning Machine (KELM) model
        model = KELMModel(layer)

        # Define a cross-validation strategy
        cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)

        # Perform cross-validation to evaluate the model performance
        scores = cross_val_score(model, X_train_scaled, y_train_scaled, cv=cv, scoring='accuracy', error_score='raise')
        result.append({'kernel': name_kernel, 'func': name_func, 'score': np.mean(scores)})

ValueError: Input arrays use different devices: cpu, /job:localhost/replica:0/task:0/device:CPU:0

In [ ]:
tf.__version__

In [ ]:
df_result = pd.DataFrame(result)

In [ ]:
df_result.to_csv(f'drive/MyDrive/Colab Notebooks/Ayang/df_result_1_{num_neurons}_{n_splits}_{n_repeats}_{max_data}.csv', index=False)

In [ ]:
fig = px.bar(
    df_result.sort_values(by=['score']),
    x='kernel',
    y='score',
    color='func',
    barmode='group',
    title=f"Cross-Validatiaon 2 (Neurons={num_neurons}, Splits={n_splits}, Repeats={n_repeats}, Maxdata={max_data}.)",
)
fig.show()

In [ ]:
raise

In [ ]:
fig = px.bar(df_result.sort_values(by=['score']), x='kernel', y='score', color='func', barmode='group', title=f"Cross-Validatiaon (Neurons={num_neurons}, Splits={n_splits}, Repeats={n_repeats}, Maxdata={max_data}.)")
fig.show()

In [ ]:
fig = px.bar(df_result.sort_values(by=['score']), x='kernel', y='score', color='func', barmode='group', title=f"Cross-Validatiaon (Neurons={num_neurons}, Splits={n_splits}, Repeats={n_repeats}, Maxdata={max_data}.)")
fig.show()

In [ ]:
raise

In [ ]:
# ================= Batas Running ================= #

In [ ]:
# Print the mean accuracy score obtained from cross-validation
print(np.mean(scores))

# Fit the ELM model to the entire dataset
model.fit(X_train_scaled, y_train_scaled)

# Save the trained model to a file
model.save("Saved Models/KELM_Model.h5")

# Load the saved model from the file
model = model.load("Saved Models/KELM_Model.h5")

# Evaluate the accuracy of the model on the training data
acc = accuracy_score(model.predict(X), y)
print(acc)

### Example

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RepeatedKFold, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing

from TfELM.Layers.KELMLayer import KELMLayer
from TfELM.Models.KELMModel import KELMModel
from TfELM.Resources.Kernel import Kernel, CombinedSumKernel, CombinedProductKernel

In [ ]:
!wget https://raw.githubusercontent.com/KStruniawski/TfELM/refs/heads/main/Data/ionosphere.txt

In [ ]:
# Hyperparameters:
num_neurons = 100
n_splits = 10
n_repeats = 10

# Loading sample dataset from Data folder
path = "https://raw.githubusercontent.com/KStruniawski/TfELM/refs/heads/main/Data/ionosphere.txt"
df = pd.read_csv(path, delimiter='\t').fillna(0)
X = df.values[:, 1:]
y = df.values[:, 0]

In [ ]:
df.shape

In [ ]:
df.head()

In [ ]:
# Label encoding and features normalization
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Encode class labels to numerical values
X = preprocessing.normalize(X)  # Normalize feature vectors

In [ ]:
y[:5]

In [ ]:
X[0]

In [ ]:
# Initialize a Kernel (it can be instanced as Kernel class and its subclasses like CombinedProductKernel)
kernel = CombinedProductKernel([Kernel("rational_quadratic"), Kernel("exponential")])

# Initialize a Kernel Extreme Learning Machine (KELM) layer
layer = KELMLayer(kernel, 'mish')

# Initialize a Kernel Extreme Learning Machine (KELM) model
model = KELMModel(layer)

# Define a cross-validation strategy
cv = RepeatedKFold(n_splits=n_splits, n_repeats=n_repeats)

# Perform cross-validation to evaluate the model performance
scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy', error_score='raise')

# Print the mean accuracy score obtained from cross-validation
print(np.mean(scores))

# Fit the ELM model to the entire dataset
model.fit(X, y)

# Save the trained model to a file
model.save("Saved Models/KELM_Model.h5")

# Load the saved model from the file
model = model.load("Saved Models/KELM_Model.h5")

# Evaluate the accuracy of the model on the training data
acc = accuracy_score(model.predict(X), y)
print(acc)